# Introduction and Background

The reviews on Yelp are important sources for business owners to know the evaluations from customers. Deeply analysis of these reviews can help business owners to improve their ratings on yelp and attract more customers. Focusing on gym business, we generate a shiny app which can provides the positions in whole business, suggestions with respect to reviews and suggestions with respect to attributes for a given business. Besides, we built a model based on whole yelp dataset to predict ratings of reviews.

# Thesis Statement

Our aim is to develop a shiny app which can give owners of gym businesses information on their position in gym business, suggestions on how to improve their ratings in yelp from both aspects of reviews and attributes. Our projects are divided into three parts:
1. Explanatory data analysis: in this part, we do data cleaning for prediction model, analysis of reviews using frequency and tfidf values, analysis of stars using some interesting plots.
2. Shiny app: in this part, we split reviews into sentences, apply LDA topic model, and compute sentiment score for each sentence, then generate business summary and business scores for each topic to provide suggestions for business owners.
3. Prediction model: In this part, we constructed neural network models to make predictions. We transferred the reviews into sequences and set the same length for them in order to have the same input format. Then we trained our model based on the review training set and predicted with the review testing set.

# Explanatory Data Analysis
Please notice that, after first presentation, we improve some of our methods and here are the most updated findings.

## 1. Get gym dataset
*See code/EDA_Get gym dataset*

We first read the ‘business_train.json’, and select the business_id whose category including 'Gym'. Then we find the column 'attributes' is dictionary-structured. So, we split it and add splited attributes as separate columns. In this way, we get business_train data with attributes.
Next, we read the ‘review_train.json’, and select reviews of gym businesses by matching the business id of ‘business_train’. In this way, we get the review_train data of gym.

## 2. Data cleaning for goal 2 prediction model
*See code/EDA_Analysis of reviews wrt words.ipynb*

In this part, we mainly did text cleaning for all reviews from review_train.json, and these cleaned reviews will be used to build prediction model. Our cleaning steps are as follows:
1. Change n't into not: this is to keep the negation infomation
2. Split into words
3. Remove punctuation
4. Change numbers into words
5. Remove non-alphabetic
6. Convert to lower case
7. Remove stop words
8. Lemmatization

## 3. Analysis of reviews wrt words
*See code/EDA_Analysis of reviews wrt words.r*

After we got cleaned reviews, we analyzed the reviews from aspect of words using two methods: word frequency, tfidf value.
1. Word frequency: we generate word cloud for overall reviews and pick top 5 frequent words in reviews of different stars. We found gym appears to be the most frequent words in all reviews and all stars. Besides, some general words(like,get) also have high frequency. These information seems not useful. However, we can see from Figure 1 below, some features words like membership, machine, equipment also have high frequency. This remind us to do some future analysis on the features of gym business.
2. TF-IDF: we pick top 3 keywords in all reviews and op 3 keywords for reviews of different stars. We found that negative words general have high tfidf value in lower stars, and positive words generally appears in higher stars. But there are some exceptions, we consider this is because we did not consider relationships between words in data cleaning part. We made some improvement of our cleaning part in following analysis.


<img style="float: center;" src="../figure/word cloud.png" width="40%">

## 4. Analysis of stars
*See code/EDA_Analysis of stars.rmd*

Firstly, we calculated the average stars for each business based on the review training data set. We then plotted the distribution of stars for the gym business field. From the plot we noticed most businesses in gym field have pretty high ratings(we didn't include this plot in the summary, please refer to our figure folder if you're interested).

Then we also visulized the geography distribution of gym business in the US map to display the number of gym businesses in every city based on our data set(see figure folder). To further check the geography effects on ratings and the number of visits for gym business. We chose Cleveland (which has the largest number of gym business in our data set) as an example. We used colors to reveal stars and the size of points to reflect the number of visits indirectly (we might assume the number of reviews is generally correlated with the number of visits). In this plot, we noticed most gym business gather in main traffic road. Those located in downtown Cleveland tends to have more visits and higher ratings compared with others.

For the attributes related to the business, we removed those with high percentage of missing values. Finally we obtained 7 attributes in total. For the attributes 'ByAppointmentOnly' and 'GoodForKids', we only found clear differences in 'ByAppointmentOnly' from plot at first. While after we performed anova analysis on them, we noticed both attributes show statistically significant among the three groups.

When it comes to the parking types, we plotted stacked barplot for each parking type. And we found lot parking and street parking tend to have higher ratings. We also noticed the low cell count problem for low ratings, and we thought it might caused by the imbalance of our data. As mentioned before, the ratings in gym business field generally have pretty high ratings. For the valet parking, we checked our data and found only 4 businesses provided valet parking in our data set.

To dig more information from the review training data set. We firstly plotted the distribution of stars in reviews. We noticed most reviews gather in 5 star while there are also some 1 star reviews. For each review, we also extract its review month and review hour. Here we may assume people tends to write a review after each time they visited the business. Therefore, we decided to utilize the number of reviews to reflect the number of visits indirectly. Also, this assumption might not valid at some times, and we still need more time to perform more researches on this. We used clock plot ([1] function provided by Vincent Zoonekynd, see references[1]) to visulize the number of reviews by month and by the hour of a day. We found people tend to start working out in January and stop working out in September. For gym business, they might could provide some promotion in January to attract more customers. We also noticed people tend to work out at night from the second plot, it might be a good strategy to provide better service and some trial lessons(which is very common is real life). But we still need to exclude some effects that people tend to write reviews at night to provide final conclusion.

Finally, we'd like to use some nouns in gym to see their distributions in review stars. We calculated the mean occurrence of each word among ratings. For coach, group, environment and vibe, they have positive effects on ratings, it’ll be a good idea for gym business to improve their ratings from these parts like providing more technical coaches, group fitness and better environment. While service, trial and membership have negative effects on ratings. The gym business should provide trial lessons of high quality to improve their first impression. For people who paid for membership, they might be very harsh about the services and the gym business should provide more benefits for them.

<img style="float: center;" src="../figure/words.png" width="60%">

# Shiny App Construction
*See code/Goal1_Get business scores.ipynb, Goal1_Linear regression.rmd*

##  1.Suggestion wrt reviews

In this part, we want to solve two questions. Where is your position in whole gym business? or in other words, how many competitors you have beat. What are the suggestions to improve ratings from both aspects of reviews and attributes? 

From the preceding analysis, we had some information:
1. Features of gym business like facilities, membership and so on may convey some information. 

2. If we want to compare different business, we need to give a score of each business.

3. There are a lot of missing values in attributes data.

Taking these information into consideration, analysis is conducted with sentence as one unit. We first use LDA topic model to find the topics talked by reviews, then determine dominant topic and compute sentiment score for each sentence, finally, we compute all topic scores for each business so that we can generate features summary, see the position in whole gym business and give suggestions based on reviews for a given business. What's more, we use weighted stars of each business as response variable and topic scores as predictors to run linear regression to give quantitive views of our suggestions to business owners.

For attributes, since there are a large amount of missing values, we want to adopt GUIDE tree to give an interpretable tree so that we can give suggestion wrt attributes.

Here are detailed summary of our analysis.

1. Divide each review into sentences, create a dataframe called data_sentence with columns are review number and sentence text. The review number will help us identify each sentence. The reasons why we choose sentence not the review as our unit are that when we use reviews to build LDA model, we found the terms in each topic are overlapped. We think this is bacause every review may talked about more than one topic, LDA model cannot accurately split the terms into each topic. Using sentence as unit, we have the assumption that one sentence only has one topic. And the result shows that this change reduced the overlap a lot.

2. For each sentence, do general data cleaning steps and get a list of words as column 3 named tokenized_sentence in data_sentence. From the previous analysis and based on the future methods we use, we keep not in our dataset because this may influence the sentiment scores for each sentence.

3. For each sentence, filter nouns as a list to become column 4 named nouns_list in data_sentence, we will apply LDA model only on these nouns.

4. Create frequency matrix and tfidf matrix for nouns. In order to get rid of the general meaningless nouns, we use tfidf matrix as our input of LDA model.

5. Apply LDA topic model on TFIDF matrix. After try several topic numbers, we pick 7 topics as our final lda_model topic number. We choose 7 topics because in visualization plot, only in 7 topics are the topic circles not overlapped and the terms under each topic converge to one subject. Also, 7 topics are close to our background information about gym business.

6. Based on some rules, get our topic-term list. 

We have several assumptions:

 * If a term shows in several topics, this term is talked a lot by customers, but is not helpful in distinguish topics. So we place more importance on terms which only shows in one topic when deciding the subjects of topics.
 
 * The subject of each topic is  determined by most of the terms in this topic but not all of them.
 
 * The subjects of topics should not be too far away from our background knowledge which includes environment, facilities, service, instructors, courses.
 
So when we decide the subject and terms of each topic, we have following principles:

 * The number of terms in each topic are not necessarily be the same, also they are not necessarily be the most relevant 10 terms. So what we have done is to limit terms in each topic to top 30 most relevant terms of this topic and then follow the principles we set to pick final terms of each topic.
 
 * The more topics a term shows, the less informative it is in deciding the subject.
 
 * Once we have decided the subject of one topic using 3 or 4 terms, we can select the rest terms using subject information instead of quantative measures. This is called subject-first-rule
 
 * Use the probability of each topic, we determine the subject of topic one by one. Once we have decided subjects of previous topics, when we decide that of next topic, we do not consider the terms belonging to previous subject any more.
 
 * Three main measures: subject, estimated term frequency(ETF), ratio between estimated term frequency and total frequency(RATIO)
 
Here are the detailed reasons for each topic:

 * Topic 1: 
 
    1.1 Although 'staff' has highest ETF in this topic, it also shows in topic 3,5,7, so we ignore it when deciding subject of this topic. 
    
    1.2 Notice that the following three terms 'equipment', 'machine', and 'facility' have high ETF as well as RATIO, we want to assigh Subject-Facilities to this topic.
    
    1.3 Since we have decide the subject of this topic, we pick the rest 7 terms according to mainly Subject-Facilities, and then RATIO, last ETF
    
    1.4 The final terms are ['equipment','machine','facility','weight','cardio','pool','studio','floor','wall','stuff']
    
 * Topic 2:
 
    2.1 We first decide the subject of this topic. Since 'class' and 'place' shows in five topics, we cannot use them to decide the subject. Then, considering both ETF and RATIO, we found 'training','fun','group','session','yoga', they are the ones with high ETF or high RATIO. So we decide the this topic is Subject-Courses
    
    2.2 After we determine the Subject-Courses, we want to re-evaluate the terms 'class' and 'place', we retain 'class' in our final term list.
    
    2.3 According to Subject first rule, considering ETF and RATIO, our final terms are ['class','training','fun','group','session','yoga','schdule','program']
    
 * Topic 3:
 
    3.1 We first decided the subject of this topic. Although term 'love' has highest ETF and RATIO, it is not useful for determining the subject. So we look at next term. Although 'room' has the second ETF, but its RATIO is not 100%, and it appears in two topics. We also ignore 'room'. Then We found 'service','everything','locker','customer','kid','shower',these are all the terms wrt Service and Accessories. So the assigned subject of topic 3 is Subject-Service and Accessories.
    
    3.2 After we determine the Subject-Service and Accessories, we pick terms mainly according to this subject.
    
    3.3 The final terms of topic 3 is ['service','everything','locker','customer','kid','shower','sauna','desk','change','child','girl','treatment','staff','bathroom']
    
 * Topic 4:
 
     4.1 We first decide the subject of this topic: gym is definitely the one with hightest frequency, but it is non-informative since it appears in several topics. Then, although 'year,'member','membership' appear in several topics, we still use it for two reasons. On one hand, the RATIOs of them in this topic are the high. On other hand, looking at other terms in this topic, the majority of them are describing membership. As a result,  the subject of this topic is Subject-Membership and Price.
    
     4.2 After we determine the Subject-Membership and Price, we pick terms mainly according to this subject.
     
     4.3 THe final terms of topic 3 is ['year,'member','membership','money','contract','family','deal','rate']

 * Topic 5:
    
     5.1 When we want to determine the subject of this topic. using principle 4, we want to delete the terms belonging to previous subject. And after doing so, we assign Subject-Trainer to this topic. Please notice there are some  terms describing time after we doing so.  However, the RATIOs of these terms are not the highest ones in this topic.  So we consider Subject-Time to other topics.
     
     5.2 The terms of this topic are ['trainer','instructor','help']
     
 * Topic 6:
     
     6.1 Use same rules as topic 5, the subject is Subject-Time
     
     6.2 The terms of this topic are ['time','day','month','morning','night','hour','door']
     
 * Topic 7:
     
     7.1 Use same rules as topic 5, the subject is Subject-Environment
     
     7.2 The terms of this topic are ['anything','music','planet','house']
     
     
We should admit that the LDA topic model may not be the best one, because we did not do Grid Search for all the parameters due to limit of memory. So there may be some misclassified terms crossing topics. As a result, after assign subject and pick the terms list for each topic. We research all the terms and the relevant topic's subject, we add some unused term to the final term list.

7. Based on the subject of each topic, enlarge the topic-term list using background infomation.

8. Assign dominant topic to each sentence: Here we assume there must be a topic talked by each sentence. 

   At first, we want to use the result from lda model, this would give each sentence a most possible topic. However, combining our practice and infomation from internet, we found the gensim lda model cannot give accurate results. 
   
   Thus, we decide to detect the terms in each sentence to decide which topic it belongs to. For each sentence, and each topic, we search the nouns_list and to see if it contains terms of this topic, then record the number of terms this sentence contains for each topic, and choose the topic whose terms shows most in this sentence.
   
   There occurs a problem, more than one topic's terms may show the same times. Under such case, we just choose the topic with smaller index because the smaller the index is, the possible the topic is. This method also has another problem. Since we have picked topic and terms manually, there are some sentences which may not contain any terms of all topics. Here we define another topic indexed 7 called 'Others' to give a certain topic to these sentences. 

9. Then we need to calculate sentiment score for each sentence. We did this using dictionary. At this step, we define seven dictionary: positive-words-dictionary, negative-words-dictionary([2]Minqing Hu and Bing Liu,see references[2]), deny-words-dictionary([3] Hutto, C.J. & Gilbert, see reference [3]), degree adverb dictionaries([4]Hownet, see reference [4]) including most-dictionary,very-dictionary,more-dictionary, ish-dictionary. 

10. Using all the dictionaries, for each sentence, we loop every word in tokenized_sentence, and search our dictionaries to find if it is positive or negative or others.If it is in either positive or negative dictionary, we call it sentiment word.

    We assume the initial score for each positive word to be 1 and negative word to be -1. Then we search the preceding words to find the degree adverbs. If we detect degree adverbs, the score for this sentiment word will time the relevant weight. We also look for deny words, it the number of deny word is odd, our score will be inversed. 
    
    Then we find next sentiment word, evaluate the words between two sentiment words, and do the same thing to calculate score for this sentiment word. For each sentiment word, we have positive score and negative score.
    
    Finally, for each sentence, we calculate the average positive score and negative score, and the final score for this sentence is just the difference between these two scores. We also add this sentiment scores as another column in our data_sentence data frame.
    
    The general idea of this computation is from reference[5], but we do improvement and modifications to apply to our dataset.

11. We then generate feature summary and compute topic scores and  weighted_stars for each business.

    Get_business_summary summary generates feature summary for given business. Get_business_score just generates eight topic scores for a give business. get_business_weighted_stars generates weighted stars for given business. The weight for each review is number of terms mentioned by this review. Because we assume that the more topic terms mentioned by a review, the valuable this review is. 
    
    We use this function to create data frame with columns business_id, eight topic scores, weighted stars, and summary data. It is saved as business_scores.csv in our data folder. This data file is the input of our shiny app.  

12. After getting all scores on each topic for all business, we can get distribution of scores on different topics. So we can output the position of given business.  Also, we can generate feature summary for given business. 

13. However, in order to give quantative and actionable suggestions on improving ratings, we want to do some statistical analysis with respect to reviews. For each business, we use weighted average stars of reviews as our response variable with weight equal to the number of feature words in each review. And 7 topic scores as our predictors to do linear regression. As a result, we can get quantative interpretation on which topic is more important, and help owners to pick the most efficient way to achieve their goal. Details of regression can be found in code/Goal1_Linear regression.rmd.


## 2. Suggestions wrt attributes
*See code/Goal1_Attributes.rmd*

After we processed the business data in gym field, we noticed most attributes have pretty high percentage of missing values. It'll be very hard for us to perform regular regression models on such data set. We then decided to use decision tree model to find potential information among attributes in influencing ratings.

Rather calculating mean stars for each business as before, we proposed weighted stars here. We selected some important words generated by LDA model and some background knowledge. We'd like to put more weights on reviews that contain larger number of important words. Then we assigned each business into 1,2,3,4,5 classes based on their weighted stars. Next, we'll perform classification decision tree based on these classes and business attributes.

We leveraged GUIDE([6]Loh, see reference [6]) to perform classification desicion tree model. The top3 important attributes are 'GoodForKids', 'BusinessAcceptsCreditCards' and 'ByAppointmentOnly'.

Our final decision tree model is displayed as below, we noticed gym business tends to get lower ratings if their above 3 attributes are missing. We think these attributes could help people find a gym suitable for their needs. If the gym business could provide these information, people can easily find a gym that meets their needs. While some unpleasant things could occur without these information, which might cause low stars in reviews.

For gym business which only accepts customers by appointment, they tend to have higher ratings. These businesses generally provide better service in many ways than others. For gym business which is good for kids, people don't need to worry about their kids and therefore has better experience in such gym. Finally, those provide street parking can facilitate people in find a parking spot and generally have higher ratings.

<figure class="half">
    <img style="float: center;" src="../figure/score.png" width="40%">
    <img style="float: center;" src="../figure/decision_tree.png" width="40%">
</figure>

# Prediction Model
*See code/Goal2_Prediction Model*

For goal 2 in predicting stars for reviews, we tried multiple methods in constructing models. We finally chose convolutional neural network as our final model, which is commonly used in natural language process. While the procedure is very time consuming and memory consuming, we didn't modify our parameters quite often after it reaches the minimal goal. Our model results on the testing data set are finally 0.68146(public) and 0.68566(private) in the Kaggle leadboard.

# Conclusion

1. We have created a shiny app for gym business owners. In this app, all the users need to do is just type in their business_id and they can get positions of their business in whole gym business on each topic. This may help them know the meaning of ratings detailed and konw which aspects of their business need to be improved. This suggestion is also supplemented by feature summary which can give a detailed description on the reviews of this business. The user can also know how much of rating they can improve if they increase the score on each topic by one unit, this will help them find the most efficient way to improve rating. 

2. Some general suggestions about topic scores are as follows:  
   When all other topic scores are fixed, weighted ratings will increase 0.8225 if score of Facilities increases by 1.  
   When all other topic scores are fixed, weighted ratings will increase 1.0843 if score of Courses increases by 1.   
   When all other topic scores are fixed, weighted ratings will increase 0.7164 if score of Service_and_Accessories increases by 1.  
   When all other topic scores are fixed, weighted ratings will increase 1.1960 if score of Membership_and_Price increases by 1.  
   When all other topic scores are fixed, weighted ratings will increase 0.6984 if score of Trainer increases by 1.  
   When all other topic scores are fixed, weighted ratings will increase 0.8813 if score of Time increases by 1.  
   When all other topic scores are fixed, weighted ratings will increase 0.6448 if score of Environment increases by 1. 
  
3. About the attributes, it's better for gym business to provide their information about 'GoodForKids', 'BusinessAcceptsCreditCards' and 'ByAppointmentOnly' on Yelp. This could help customers to find the best gym that fits their needs and improve the service ratings. The gym business could improve ratings by setting environment more suitable for kids and provide street parking. If possible, the gym business could provide better services by accepting appointment only.

# Advantages and Disadvantages

Advantages:

1. Our method is flexiable and business specific. Although we give some general suggestion that all the gym business can use, we generate  business specific postion and feature summary for business owners.
2. Our tree method on attributes takes missing values into consideration. This helps us aviod imputation, also problems brought by imputation.
    
Disadvantages:

1. The LDA model we used may not be the best model since we did not do grid search.
2. The sentiment score computation method may not that accurate.
3. When assigning dominant topics, maybe we should use another module so that we can directly get dominant topic from our model.
4. The number of reviews of each business  is not large enough.
5. We only considered attributes as predictor variables in providing potential objects that could influence the ratings. While this situation might be very complicated, there might be multiple reasons behind attributes. We can't provide more insightful suggestions based on each business.
6. For the purpose of this project, we just provide suggestions that might improve ratings. While in real business, ratings might not be a main concern for business owner. Profits are what owners care in general, but it can't be improved simply from improving stars. We still need to do more research to provide useful information for business owners.

# Contributions

Wanwan Su: Get gym datasets, shiny app, jupyter notebook and some datasets generation.

Yunhui Qi: Analysis of reviews wrt words. Goal 1 except for anlysis of attributes. Goal 2 data cleaning, shiny app. jupyter notebook.

Qiming Hao: Analysis of stars. Goal 1 analysis of attributes. Goal 2 model construnction. jupyter notebook.

# References

[1] http://www.r-graph-gallery.com/49-clock-plot/ 

[2] Minqing Hu and Bing Liu. "Mining and Summarizing Customer Reviews." Proceedings of the ACM SIGKDD International Conference on Knowledge Discovery and Data       Mining (KDD-2004), Aug 22-25, 2004, Seattle, Washington, USA.

[3] Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text. Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.

[4] Hownet sentiment degree adverbs dictionary.

[5] https://www.jianshu.com/p/6826672a379c

[6] Loh, W.-Y. (2011), Classification and regression trees, Wiley Interdisciplinary Reviews: Data Mining and Knowledge Discovery, vol.1, 14-23.